# Imports

In [2]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests

# Constants and Funcs

In [3]:
fre_mulitplier = 78.5

countries = ['bangladesh', 'brazil', 'cambodia','india', 'china', 'indonesia', 'mexico']

city_country = ['brazil', 'cambodia','rwanda']

div_admin_countries = ['bangladesh']

def convert_to_flt(str):
    try:
        return float(str.replace(",", ""))  
    except ValueError:
        return None

In [4]:
def make_df_admin(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    # print(soup.prettify())

    rows = soup.find_all('tr', class_=['rname', 'admin2'])
    # rows = soup.find_all('tr')

    data = []

    header_cells = soup.find_all('th', class_='rpop')
    # import pdb; pdb.set_trace()
    years = []
    for header in header_cells:
        # Extract the year from the 'span' element inside the 'th' (e.g., "2005-06-30")
        span = header.find('span', class_='unit')
        # import pdb; pdb.set_trace()
        if span:
            # Extract year from the text, assuming it's in the form "Census\nYYYY-MM-DD"
            year = span.text.strip().split('-')[0][-4:]  # Get the year part
            years.append(year)


        
    for row in rows:
        try:
            # Get the name, status, and population for each census year
            name = row.find('td', class_='rname').text.strip()
            status = row.find('td', class_='rstatus').text.strip()

            # Extract area and population data for different census years
            # import pdb; pdb.set_trace()
            area = convert_to_flt(row.find('td', class_='rname')['data-area'])  # Get the 'data-area' attribute

            # Extract all population data (there could be multiple columns for different years)
            population_cells = row.find_all('td', class_='rpop')

            # Create a dictionary to store population data for each year
            population_data = {}
            for i, cell in enumerate(population_cells):
                population = cell.text.strip()
                population_data[f'Population {years[i]}'] = convert_to_flt(population)

            # Append the extracted data into the list
            row_data = [name, status, area] + list(population_data.values())
            data.append(row_data)
            
        except:
            continue
    # Create a DataFrame
    columns = ['Name', 'Status', 'Area'] + [year for year in years]  
    df = pd.DataFrame(data, columns=columns)

    latest_population_column = years[-1]

    return df, latest_population_column

In [5]:
def make_df_city(page, section_id):
    soup = BeautifulSoup(page.content, 'html.parser')
    soup = soup.find(id=section_id)
    # print(soup.prettify())

    # rows = soup.find_all('tr', class_=['rname', 'admin2'])
    rows = soup.find_all('tr')

    data = []

    header_cells = soup.find_all('th', class_='rpop')

    years = []

    for header in header_cells:
        # Extract the year from the 'span' element inside the 'th' (e.g., "2005-06-30")
        span = header.find('span', class_='unit')
        if span:
            # Extract year from the text, assuming it's in the form "Census\nYYYY-MM-DD"
            year = span.text.strip().split('-')[0][-4:]  # Get the year part
            if year not in years:
                years.append(year)

    # import pdb; pdb.set_trace()
        
    for row in rows:
        if section_id == 'adminareas':
            try:
                # Get the name, status, and population for each census year
                name = row.find('td', class_='rname').text.strip()
                status = row.find('td', class_='rstatus').text.strip()

                # Extract area and population data for different census years
                # import pdb; pdb.set_trace()
                area = row.find('td', class_='rarea').text.strip()  # Get the 'data-area' attribute

                # Extract all population data (there could be multiple columns for different years)
                population_cells = row.find_all('td', class_='rpop')

                # Create a dictionary to store population data for each year
                population_data = {}
                for i, cell in enumerate(population_cells):
                    population = cell.text.strip()
                    population_data[f'Population {years[i]}'] = convert_to_flt(population)

                # Append the extracted data into the list
                row_data = [name, status, area] + list(population_data.values())
                data.append(row_data)
                
            except Exception as e:
                # import pdb; pdb.set_trace()
                continue
            # Create a DataFrame
          

        elif section_id == "largecities":
            # import pdb; pdb.set_trace()
            try:
                # Get the name, status, and population for each census year
                name = row.find('td', {'class': None}).text.strip()
                # Extract all population data (there could be multiple columns for different years)
                population_tag = row.find('td', {'class': 'rpop'}).text.strip()

                # Append the extracted data into the list
                row_data = [name, population_tag]
                data.append(row_data)
                
            except Exception as e:
                # import pdb; pdb.set_trace()
                continue
            # Create a DataFrame

        elif section_id == "citysection":
            # import pdb; pdb.set_trace()
            try:
                # Get the name, status, and population for each census year
                name = row.find('td', class_='rname').text.strip()
                # status = row.find('td', class_='rstatus').text.strip()

                # Extract area and population data for different census years
                # import pdb; pdb.set_trace()
                area = row.find('td', class_='noviz').text.strip()  # Get the 'data-area' attribute

                # Extract all population data (there could be multiple columns for different years)
                population_cells = row.find_all('td', class_='rpop')

                # Create a dictionary to store population data for each year
                population_data = {}
                for i, cell in enumerate(population_cells):
                    population = cell.text.strip()
                    population_data[f'Population {years[i]}'] = convert_to_flt(population)

                # Append the extracted data into the list
                row_data = [name, area] + list(population_data.values())
                data.append(row_data)
                
            except Exception as e:
                # import pdb; pdb.set_trace()
                continue
            # Create a DataFrame

        
    
    if section_id == 'adminareas':
        columns = ['Name', 'Status', 'Area'] + [year for year in years]  
    elif section_id == 'largecities':
        columns = ['Name'] + [year for year in years] 
    elif section_id == 'citysection':
        columns = ['Name', 'Area'] + [year for year in years] 

    df = pd.DataFrame(data, columns=columns)

    latest_population_column = years[-1]

    return df, latest_population_column



In [6]:
def make_df_blank(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    # print(soup.prettify())

    # rows = soup.find_all('tr', class_=['rname', 'admin2'])
    admin_section = soup.find('section', id='adminareas')
    rows = admin_section.find_all('tr', attrs={'itemscope': True})
    # rows = soup.find_all('tr')

    data = []

    header_cells = admin_section.find_all('th', class_='rpop')
    # import pdb; pdb.set_trace()
    years = []
    for header in header_cells:
        # Extract the year from the 'span' element inside the 'th' (e.g., "2005-06-30")
        span = header.find('span', class_='unit')
        # import pdb; pdb.set_trace()
        if span:
            # Extract year from the text, assuming it's in the form "Census\nYYYY-MM-DD"
            year = span.text.strip().split('-')[0][-4:]  # Get the year part
            years.append(year)

    years = list(dict.fromkeys(years))  # Remove duplicates while preserving order

    for row in rows:
        # import pdb; pdb.set_trace()
        try:
            # Get the name, status, and population for each census year
            name = row.find('td', class_='rname').find('span', itemprop='name').text.strip()

            status = row.find('td', class_='rname')['data-status'].strip()

            # Extract area and population data for different census years
            # import pdb; pdb.set_trace()
            area_text = row.find('td', class_='rarea').text.strip()  # Get the 'data-area' attribute
            area = convert_to_flt(area_text) 
            # Extract all population data (there could be multiple columns for different years)
            population_cells = row.find_all('td', class_='rpop')
            population_data = {}
            for i, cell in enumerate(population_cells):
                population = cell.text.strip()
                population_data[f'Population {years[i]}'] = convert_to_flt(population)

            # Append the full row data
            row_data = [name, status, area] + list(population_data.values())
            data.append(row_data)
            
        except:
            continue

    # import pdb; pdb.set_trace()
    # Create a DataFrame
    columns = ['Name', 'Status', 'Area'] + [year for year in years]  
    df = pd.DataFrame(data, columns=columns)

    latest_population_column = years[-1]

    return df, latest_population_column

In [17]:
def make_df_l2(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table')
    if table is None:
        raise ValueError("No table found on the page.")

    # Extract table headers
    headers = []
    for th in table.find_all('th'):
        headers.append(th.get_text(strip=True))

    # Extract table rows
    rows = []
    for tr in table.find_all('tr')[1:]:  # Skip the header row
        cells = tr.find_all(['td', 'th'])
        row = [cell.get_text(strip=True) for cell in cells]
        if row:
            rows.append(row)

    # header = rows[0][1:]
    new_header = ['L1', 'L2'] + headers[1:]

    data = []

    state = rows[0][0]

    if len(rows) == 1:
        data.append([state, state] + rows[0][1:])
    else:
        for row in rows[1:]:
            if len(row) >= 2:  # make sure there's something to split
                new_row = [state] + [row[0]] + row[1:]
                data.append(new_row)

            
    
    

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(data, columns=new_header)
    df = df.iloc[:-1]

    return df

In [8]:
def make_df_brazil_problem(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table')
    if table is None:
        raise ValueError("No table found on the page.")

    # Get header
    header_cells = table.find('tr').find_all('th')
    headers = [cell.get_text(strip=True) for cell in header_cells if cell.get_text(strip=True)]

    rows = []
    for tr in table.find_all('tr')[1:]:  # Skip the header
        cells = tr.find_all(['td'])
        row = [cell.get_text(strip=True) for cell in cells]
        if row:
            row = [row[0]] + row
            rows.append(row)

    
    headers = ['L1', 'L2'] + headers[1:]

    
    # Auto-adjust column count
    max_len = max(len(row) for row in rows)
    if len(headers) < max_len:
        # Fill in missing column names as "Extra_1", "Extra_2", etc.
        headers += [f"Extra_{i+1}" for i in range(max_len - len(headers))]
    elif len(headers) > max_len:
        # Trim headers if too many
        headers = headers[:max_len]

    df = pd.DataFrame(rows, columns=headers)
    df = df.iloc[:-1]
    return df

# Webscraper

## For Asia and LATAM

In [ ]:
for country in countries:
    print(country)
    if country in city_country:
        page = requests.get(f"https://www.citypopulation.de/en/{country}/cities/")
        admin_df, year = make_df_city(page, 'adminareas')
        # city_df, year = make_df_city(page, 'largecities')
        # city_section_df, year = make_df_city(page, 'citysection')

        # df = pd.concat([admin_df, city_df, city_section_df], axis=0)
        df = admin_df

    elif country in div_admin_countries:
        page = requests.get(f"https://www.citypopulation.de/en/{country}/div/admin/")
        df, year = make_df_blank(page)
    
    else:
        page = requests.get(f"https://www.citypopulation.de/en/{country}/admin/")
        df, year = make_df_admin(page)

        # country_admin_df_name = f"{country}_admin_df"
        # country_city_df_name = f"{country}_city_df"
        # country_city_section_df_name = f"{country}_city_section_df"
        # globals()[country_admin_df_name] = admin_df
        # globals()[country_city_df_name] = city_df
        # globals()[country_city_section_df_name] = city_section_df

# import pdb; pdb.set_trace()
    country_df_name = f"{country}_df"  
    country_year_name = f"{country}_year"
    globals()[country_df_name] = df
    globals()[country_year_name] = year

    


bangladesh


NameError: name 'make_df_div' is not defined

## For Africa

In [ ]:
page = requests.get(f"https://www.citypopulation.de/en/kenya/sub/admin/")
kenya_df, year = make_df_admin(page)
kenya_df

,Name,Status,Area,2009,2019
0,Baringo,County,10976.4,555561.0,666763.0
1,Bomet,County,2530.9,730129.0,875689.0
2,Bungoma,County,3023.9,1375063.0,1670570.0
3,Busia,County,1696.3,743946.0,893681.0
4,Elgeyo-Marakwet,County,3032.0,369998.0,454480.0
5,Embu,County,2820.7,516212.0,608599.0
6,Garissa,County,44736.0,623060.0,841353.0
7,Homa Bay,County,3152.5,963794.0,1131950.0
8,Isiolo,County,25350.6,143294.0,268002.0
9,Kajiado,County,21871.1,687312.0,1117840.0


In [ ]:
page = requests.get(f"https://www.citypopulation.de/en/liberia/")
liberia_df, year = make_df_blank(page)
liberia_df

,Name,Status,Area,1984,2008,2022
0,Bomi,Cnty,1942.0,66420.0,84119.0,133705.0
1,Bong,Cnty,8769.0,255813.0,333481.0,467561.0
2,Gbarpolu,Cnty,9685.0,48399.0,83388.0,95995.0
3,Grand Bassa,Cnty,7932.0,159648.0,221693.0,293689.0
4,Grand Cape Mount,Cnty,5160.0,79322.0,127076.0,178867.0
5,Grand Gedeh,Cnty,10480.0,63028.0,125258.0,216692.0
6,Grand Kru,Cnty,3894.0,62791.0,57913.0,109342.0
7,Lofa,Cnty,9978.0,199242.0,276863.0,367376.0
8,Margibi,Cnty,2615.0,151792.0,209923.0,304946.0
9,Maryland,Cnty,2296.0,69267.0,135938.0,172587.0


In [ ]:
page = requests.get(f"https://www.citypopulation.de/en/rwanda/admin/")
rwanda_df, year = make_df_admin(page)
rwanda_df

,Name,Status,Area,2002,2012,2022
0,Est (Intara y'Iburasirazuba) [Eastern],Province,9458.0,1700137.0,2595703.0,3563145.0
1,Kigali,City,730.0,765325.0,1132686.0,1745555.0
2,Nord (Intara y'Amajyaruguru) [Northern],Province,3276.0,1560862.0,1726370.0,2038511.0
3,Ouest (Intara y'Iburengerazuba) [Western],Province,5882.0,2043555.0,2471239.0,2896484.0
4,Sud (Intara y'Amajyepfo) [Southern],Province,5963.0,2058674.0,2589975.0,3002699.0


In [ ]:
page = requests.get(f"https://www.citypopulation.de/en/southafrica/admin/")
southafrica_df, year = make_df_admin(page)
southafrica_df

,Name,Status,Area,1996,2001,2011,2022
0,Eastern Cape,Province,168966.0,6147244.0,6278651.0,6562053.0,7230204.0
1,Free State (Oranje Free State),Province,129825.0,2633504.0,2706775.0,2745590.0,2964412.0
2,Gauteng,Province,18178.0,7834620.0,9390528.0,12272263.0,15099422.0
3,KwaZulu-Natal,Province,94361.0,8572302.0,9584129.0,10267300.0,12423907.0
4,Limpopo (Northern Transvaal),Province,125754.0,4576133.0,4995462.0,5404868.0,6572721.0
5,Mpumalanga (Eastern Transvaal),Province,76495.0,3124203.0,3365957.0,4039939.0,5143324.0
6,Northern Cape,Province,372889.0,1011864.0,991876.0,1145861.0,1355945.0
7,North West,Province,104882.0,2726828.0,2982064.0,3509953.0,3804548.0
8,Western Cape,Province,129462.0,3956875.0,4524335.0,5822734.0,7433020.0


In [ ]:
page = requests.get(f"https://www.citypopulation.de/en/gambia/admin/")
gambia_df, year = make_df_admin(page)
gambia_df

,Name,Status,Area,1993,2003,2013
0,Banjul,Local Government Area,12.0,42326.0,35061.0,31054.0
1,Basse (Upper River),Local Government Area,2070.0,155059.0,182586.0,237220.0
2,Brikama (Western),Local Government Area,1764.0,234917.0,389594.0,688744.0
3,Janjanbureh,Local Government Area,1428.0,88247.0,107212.0,125204.0
4,Kanifeng,Local Government Area,76.0,228214.0,322735.0,377134.0
5,Kerewan (North Bank),Local Government Area,2255.0,156462.0,172835.0,220080.0
6,Kuntaur,Local Government Area,1467.0,67774.0,78491.0,96703.0
7,Mansakonko (Lower River),Local Government Area,1618.0,65146.0,72167.0,81042.0


In [ ]:
page = requests.get(f"https://www.citypopulation.de/en/zambia/admin/")
zambia_df, year = make_df_admin(page)
zambia_df

,Name,Status,Area,2000,2010,2022
0,Central,Province,94394.0,1012257.0,1307111.0,2261336.0
1,Copperbelt,Province,31328.0,1581221.0,1972317.0,2768192.0
2,Eastern,Province,68948.8,1306173.0,1696555.0,2462682.0
3,Luapula,Province,50567.0,775353.0,991927.0,1519478.0
4,Lusaka,Province,21986.0,1391329.0,2191225.0,3093617.0
5,Muchinga,Province,70243.2,449296.0,607763.0,922213.0
6,Northern,Province,77650.0,809400.0,1105824.0,1623853.0
7,North-Western,Province,125826.0,583350.0,727044.0,1278357.0
8,Southern,Province,85283.0,1212124.0,1589926.0,2388091.0
9,Western,Province,126386.0,765088.0,902974.0,1375604.0


In [ ]:
page = requests.get(f"https://www.citypopulation.de/en/malawi/admin/")
malawi_df, year = make_df_admin(page)
malawi_df

,Name,Status,Area,1977,1987,1998,2008,2018,2023
0,Central,Region,35641.0,2143716.0,3110986.0,4066340.0,5497252.0,7523340.0,8533879.0
1,Northern,Region,27131.0,648853.0,911787.0,1233560.0,1679491.0,2289780.0,2532270.0
2,Southern,Region,31780.0,2754891.0,3965734.0,4633968.0,5852755.0,7750629.0,8743362.0


In [ ]:
page = requests.get(f"https://www.citypopulation.de/en/ghana/admin/")
ghana_df, year = make_df_admin(page)
ghana_df

,Name,Status,Area,2010,2021
0,Ahafo (← Brong Ahafo),Region,5196.0,484210.0,564668.0
1,Ashanti,Region,24389.0,4780380.0,5440463.0
2,Bono (← Brong Ahafo),Region,11113.0,922617.0,1208649.0
3,Bono East (← Brong Ahafo),Region,23248.0,904156.0,1203400.0
4,Central,Region,9826.0,2201863.0,2859821.0
5,Eastern,Region,19323.0,2633154.0,2925653.0
6,Greater Accra,Region,3245.0,4010054.0,5455692.0
7,North East (← Northern),Region,9070.0,465005.0,658946.0
8,Northern,Region,26524.0,1544946.0,2310939.0
9,Oti (← Volta),Region,11066.0,636889.0,747248.0


In [ ]:
page = requests.get(f"https://www.citypopulation.de/en/eswatini/")
eswatini_df, year = make_df_blank(page)
eswatini_df

,Name,Status,Area,1976,1986,1997,2007,2017,2024
0,Hhohho,Reg,3569.0,95759.0,178936.0,255455.0,282734.0,320651.0,355700.0
1,Lubombo,Reg,5947.0,81800.0,153958.0,194323.0,207731.0,212531.0,237500.0
2,Manzini,Reg,4068.0,101277.0,192596.0,280972.0,319530.0,355945.0,397000.0
3,Shiselweni,Reg,3779.0,95735.0,155569.0,198978.0,208454.0,204111.0,212200.0


## Brazil l2

In [ ]:
page = requests.get(f"https://www.citypopulation.de/en/brazil/regiaonorte/admin/12__acre/")
arce_df = make_df_l2(page)
arce_df

,L1,Acre,State,"417,718","557,526","733,559","830,018","880,631",
0,Acre,Acrelândia,Municipality,"5,467","7,935","12,538","14,021","14,657",→
1,Acre,Assis Brasil,Municipality,"2,917","3,490","6,072","8,100","8,573",→
2,Acre,Brasiléia,Municipality,"13,992","17,013","21,398","26,000","27,841",→
3,Acre,Bujari,Municipality,"3,182","5,826","8,471","12,917","13,766",→
4,Acre,Capixaba,Municipality,"2,316","5,206","8,798","10,392","10,922",→
5,Acre,Cruzeiro do Sul,Municipality,"47,812","67,441","78,507","91,888","98,382",→
6,Acre,Epitaciolândia,Municipality,"7,429","11,028","15,100","18,757","19,739",→
7,Acre,Feijó,Municipality,"17,769","26,722","32,412","35,426","37,644",→
8,Acre,Jordão,Municipality,"4,210","4,454","6,577","9,222","9,787",→
9,Acre,Mâncio Lima,Municipality,"7,842","11,095","15,206","19,294","20,329",→


In [ ]:
page = requests.get(f"https://www.citypopulation.de/en/brazil/regiaonordeste/admin/27__alagoas/")
alagoas_df = make_df_l2(page)
alagoas_df

,L1,L2,State,"2,514,100","2,822,621","3,120,887","3,127,683","3,220,104",
0,Alagoas,Água Branca,Municipality,"18,157","18,660","19,389","19,008","19,550",→
1,Alagoas,Anadia,Municipality,"16,337","17,849","17,318","13,966","14,193",→
2,Alagoas,Arapiraca,Municipality,"164,921","186,466","213,671","234,696","243,661",→
3,Alagoas,Atalaia,Municipality,"38,563","40,552","44,326","37,512","38,530",→
4,Alagoas,Barra de Santo Antônio,Municipality,"7,423","11,351","14,230","16,365","16,735",→
...,...,...,...,...,...,...,...,...,...
97,Alagoas,Taquarana,Municipality,"16,809","17,046","18,615","19,032","19,422",→
98,Alagoas,Teotônio Vilela,Municipality,"29,664","36,881","41,152","38,053","39,161",→
99,Alagoas,Traipu,Municipality,"22,680","23,439","25,992","23,565","24,124",→
100,Alagoas,União dos Palmares,Municipality,"57,425","58,620","62,326","59,280","60,874",→


In [10]:
page = requests.get(f"https://www.citypopulation.de/en/brazil/regiaonorte/admin/16__amap%C3%A1/")
amapá_df = make_df_l2(page)
amapá_df

,L1,L2,State,"289,397","477,032","669,526","733,759","802,837",
0,Amapá,Amapá,Municipality,"6,655","7,121","8,069","7,943","8,434",→
1,Amapá,Calçoene,Municipality,"5,177","6,730","9,000","10,612","11,391",→
2,Amapá,Cutias,Municipality,"1,676","3,280","4,696","4,461","4,725",→
3,Amapá,Ferreira Gomes,Municipality,"2,350","3,562","5,802","6,666","7,145",→
4,Amapá,Itaubal,Municipality,"1,314","2,894","4,265","5,599","6,043",→
5,Amapá,Laranjal do Jari,Municipality,"16,772","28,515","39,942","35,114","37,969",→
6,Amapá,Macapá,Municipality,"168,225","283,308","398,204","442,933","487,200",→
7,Amapá,Mazagão,Municipality,"8,911","11,986","17,032","21,924","23,575",→
8,Amapá,Oiapoque,Municipality,"7,555","12,886","20,509","27,482","30,481",→
9,Amapá,Pedra Branca do Amapari,Municipality,"1,460","4,009","10,772","12,847","13,798",→


In [76]:

page = requests.get(f"https://www.citypopulation.de/en/brazil/regiaocentrooeste/admin/distrito_federal/5300108__bras%C3%ADlia/")
test_df = make_df_brazil_problem(page)
test_df

,L1,L2,Status,PopulationCensus1991-09-01,PopulationCensus2000-08-01,PopulationCensus2010-08-01,PopulationCensus2022-08-01,PopulationEstimate2024-07-01,Extra_1
0,Brasília,Brasília,Municipality,"1,601,094","2,051,146","2,572,159","2,817,381","2,982,818",


In [12]:
brazil_sites = ['https://www.citypopulation.de/en/brazil/regiaonorte/admin/12__acre/', 
                'https://www.citypopulation.de/en/brazil/regiaonordeste/admin/27__alagoas/',
                'https://www.citypopulation.de/en/brazil/regiaonorte/admin/16__amap%C3%A1/',
                'https://www.citypopulation.de/en/brazil/regiaonorte/admin/13__amazonas/',
                'https://www.citypopulation.de/en/brazil/regiaonordeste/admin/29__bahia/',
                'https://www.citypopulation.de/en/brazil/regiaonordeste/admin/23__cear%C3%A1/',
                'https://www.citypopulation.de/en/brazil/regiaosudeste/admin/32__esp%C3%ADrito_santo/',
                'https://www.citypopulation.de/en/brazil/regiaosudeste/admin/31__minas_gerais/',
                'https://www.citypopulation.de/en/brazil/regiaonorte/admin/15__par%C3%A1/',
                'https://www.citypopulation.de/en/brazil/regiaonordeste/admin/25__para%C3%ADba/',
                'https://www.citypopulation.de/en/brazil/regiaosud/admin/41__paran%C3%A1/',
                'https://www.citypopulation.de/en/brazil/regiaonordeste/admin/26__pernambuco/',
                'https://www.citypopulation.de/en/brazil/regiaonordeste/admin/22__piau%C3%AD/',
                'https://www.citypopulation.de/en/brazil/regiaosudeste/admin/33__rio_de_janeiro/',
                'https://www.citypopulation.de/en/brazil/regiaonordeste/admin/24__rio_grande_do_norte/',
                'https://www.citypopulation.de/en/brazil/regiaosud/admin/43__rio_grande_do_sul/',
                'https://www.citypopulation.de/en/brazil/regiaonorte/admin/11__rond%C3%B4nia/',
                'https://www.citypopulation.de/en/brazil/regiaonorte/admin/14__roraima/',
                'https://www.citypopulation.de/en/brazil/regiaosud/admin/42__santa_catarina/',
                'https://www.citypopulation.de/en/brazil/regiaosudeste/admin/35__s%C3%A3o%20paulo/',
                'https://www.citypopulation.de/en/brazil/regiaonordeste/admin/28__sergipe/',
                'https://www.citypopulation.de/en/brazil/regiaonorte/admin/17__tocantins/']

brazil_problem_sites =['https://www.citypopulation.de/en/brazil/regiaocentrooeste/admin/distrito_federal/5300108__bras%C3%ADlia/',
                       ]

brazil_df = pd.DataFrame()

for site in brazil_sites:
    page = requests.get(site)
    df = make_df_l2(page)
    df = df[['L1', 'L2']]
    brazil_df = pd.concat([brazil_df, df], axis=0)

for site in brazil_problem_sites:
    page = requests.get(site)
    df = make_df_brazil_problem(page)
    df = df[['L1', 'L2']]
    brazil_df = pd.concat([brazil_df, df], axis=0)

brazil_df

,L1,L2
0,Acre,Acrelândia
1,Acre,Assis Brasil
2,Acre,Brasiléia
3,Acre,Bujari
4,Acre,Capixaba
...,...,...
135,Tocantins,Tupirama
136,Tocantins,Tupiratins
137,Tocantins,Wanderlândia
138,Tocantins,Xambioá


# Excel Writer

In [ ]:
# with pd.ExcelWriter('data/all_citypopulation_data.xlsx', engine='openpyxl') as writer:
#     bangladesh_df.to_excel(writer, sheet_name='Bangladesh', index=False)
#     brazil_df.to_excel(writer, sheet_name='Brazil', index=False)
#     cambodia_df.to_excel(writer, sheet_name='Cambodia', index=False)
#     china_df.to_excel(writer, sheet_name='China', index=False)
#     india_df.to_excel(writer, sheet_name='India', index=False)
#     indonesia_df.to_excel(writer, sheet_name='Indonesia', index=False)
#     mexico_df.to_excel(writer, sheet_name='Mexico', index=False)
#     kenya_df.to_excel(writer, sheet_name='Kenya', index=False)
#     liberia_df.to_excel(writer, sheet_name='Liberia', index=False)
#     rwanda_df.to_excel(writer, sheet_name='Rwanda', index=False)
#     southafrica_df.to_excel(writer, sheet_name='South Africa', index=False)
#     gambia_df.to_excel(writer, sheet_name='Gambia', index=False)
#     zambia_df.to_excel(writer, sheet_name='Zambia', index=False)
#     malawi_df.to_excel(writer, sheet_name='Malawi', index=False)
#     ghana_df.to_excel(writer, sheet_name='Ghana', index=False)
#     eswatini_df.to_excel(writer, sheet_name='Eswatini', index=False)
#     workbook = writer.book



In [ ]:
# brazil_df.to_excel('data/brazil_df.xlsx', index=False)

In [18]:
with pd.ExcelWriter('data/brazil_l2_population.xlsx', engine='openpyxl') as writer:
    for site in brazil_sites:
        page = requests.get(site)
        df = make_df_l2(page)
        state = df['L1'][0]
        df.to_excel(writer, sheet_name=state, index=False)
    workbook = writer.book